Upload File

In [ ]:

from google.colab import files
uploaded = files.upload()

Saving yelp_reviewsAZcombined.csv to yelp_reviewsAZcombined.csv


Upload Data to Dataframe

In [ ]:
import pandas as pd


In [ ]:
"""
Arizona data. Sorting to obtain the users with the most reviews. And then 
analyzing to make predictions.
"""
# Import combined csv data.
dfAZ = pd.read_csv('yelp_reviewsAZcombined.csv')

# Count number of ratings performed by each user.
ratingsCount = pd.DataFrame(dfAZ.groupby('user_id')['stars_x'].count())
ratingsCount['ratings_performed'] = dfAZ.groupby('user_id')['stars_x'].count()
ratingsCount = ratingsCount.loc[:, ['ratings_performed']]

# Organize Arizona reviews by users with most reviews to users with least reviews
ratingsCount = ratingsCount.sort_values(by='ratings_performed', ascending=False)
dfUsers = ratingsCount.merge(dfAZ, on=['user_id'])
dfUsers = dfUsers.drop(columns="Unnamed: 0")










Explore the Data

In [ ]:
dfUsers.head()

,user_id,ratings_performed,business_id,name,address,city,state,postal_code,stars_x
0,ELcQDlf69kb-ihJfxZyL0A,238,8OeHn9KCbICuRBs_XiDDUA,Buffalo Chip Saloon & Steakhouse,6823 E Cave Creek Rd,Cave Creek,AZ,85331.0,4.0
1,ELcQDlf69kb-ihJfxZyL0A,238,8OeHn9KCbICuRBs_XiDDUA,Buffalo Chip Saloon & Steakhouse,6823 E Cave Creek Rd,Cave Creek,AZ,85331.0,2.0
2,ELcQDlf69kb-ihJfxZyL0A,238,ACb637NG7EnswRs8U_BX_w,Bar Louie - Tempe Marketplace,"2000 Rio Salado Pkwy, Ste 1220",Tempe,AZ,85281.0,3.0
3,ELcQDlf69kb-ihJfxZyL0A,238,Kk6k9cs4rXfDkC9Fjv0MjQ,Jacksons Car Wash,15515 N Scottsdale Rd,Scottsdale,AZ,85260.0,3.0
4,ELcQDlf69kb-ihJfxZyL0A,238,6gFcUhaYdinTbIhO2pcvdg,Tempe Diablo Stadium,2200 W Alameda Dr,Tempe,AZ,85252.0,4.0


In [ ]:
dfUsers.describe()

,ratings_performed,postal_code,stars_x
count,307380.000000,307233.000000,307380.000000
mean,7.133372,85187.869832,3.720678
std,16.320563,299.085497,1.541244
min,1.000000,36867.000000,1.000000
25%,1.000000,85032.000000,2.000000
50%,2.000000,85234.000000,4.000000
75%,6.000000,85281.000000,5.000000
max,238.000000,92008.000000,5.000000


In [ ]:
ratings = pd.DataFrame(dfUsers.groupby('name')['stars_x'].mean())
ratings.head()

,stars_x
name,
1 800 Flowers,2.000000
1 800-Flowers,2.000000
1 Brothers Pizza,3.777778
1 One Armor,5.000000
1-800 WATER DAMAGE of Phoenix,5.000000


In [ ]:
ratings['number_of_ratings'] = dfUsers.groupby('name')['stars_x'].count()
ratings.head()

,stars_x,number_of_ratings
name,,
1 800 Flowers,2.000000,3
1 800-Flowers,2.000000,4
1 Brothers Pizza,3.777778,72
1 One Armor,5.000000,13
1-800 WATER DAMAGE of Phoenix,5.000000,10


Make user and Business rating matrix

In [ ]:
dfUsers_sample = dfUsers.head(220000)
starMatrix = dfUsers_sample.pivot_table(index='user_id', columns='name', values='stars_x')
starMatrix.head()

name,1 800 Flowers,1 800-Flowers,1 Brothers Pizza,1 One Armor,1-800 WATER DAMAGE of Phoenix,1-800-Flowers,101 Spay-Neuter & Dental Clinic,101 Towing,11 Bravo Mobile Auto Reconditioning,1133 West Fifth,1360 West Grill,1480 KPHX,16th Street Sports Bar & Grill,18|8 Fine Men's Salons - Silverstone,19th Donut Hole,1on1 Boxing Fitness,1st Choice Mechanical,1st Integrity Movers,1st Pet Veterinary Centers,1st Scottsdale Dental,1st USA Carpet Cleaning,"2 ""Di"" 4 Gourmet Karmel Korn",2 Dads Bounce Houses and Party Rentals,2 Guyes Garage,2 Wrenches Automotive,20/20 Image Eye Centers,2020 Vision Photos,210 Home Buyers Warrantee,24 Carrots Juice Bar & Cafe,24 Hour Fitness,24 Hour Glass & Door Repair,24/7 AC LLC,25 Chiropractic,25th Street Automotive,2BU Salon Spa / Aveda,2U Glass & Tint,2b Mod,3 Dudes Quilting,3 Minute Express Car Wash,3 Vintage Chicks,...,Zoyo Frozen Yogurt - Phoenix Arcadia,Zuch Periodontics,Zumiez,Zuni Hills Elementary School,a Midwife's Kaleidoscope,aka Green,ampm,bare BEAUTY Outlet,barre3 - Chandler,bebe,crunch50,d'Vine Gourmet,de la Cruz Bistro,di Sciacca,enVoqueMD,epicPHX Real Estate Group,giggle,home-ology,i9 Sports,iFixAZ,iLoveKickboxing - Glendale,iNails & Toes,iPowerWeb,iQ Abacus Math School,iRide Worldwide,iSmile Family Dentistry & Implant Surgical Center - Glendale,loanDepot,lucy Activewear,pH Balanced Pool Service,racelab,stonegrill,sunburst smoke shop,swampt,the inside coup®,the motherhood,uBreakiFix,uBreakiFix Goodyear,urbAna,west elm,zombie mfg
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
--1av6NdbEbMiuBr7Aup9A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--2HUmLkcNHZp0xw6AMBPg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--3B8LdT1NCD-bPkwS5-5g,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--4rAAfZnEIAKJE80aIiYg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--9hgcpqQPBbv0GFBQ4jsQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
ratings.sort_values('number_of_ratings', ascending=False).head(10)

,stars_x,number_of_ratings
name,,
Joe's Farm Grill,4.196908,2458
The Arrogant Butcher,3.930388,2399
Windsor,4.200308,1298
The Clever Koi,4.305753,1269
Zinburger,3.874781,1142
Oregano's Pizza Bistro,3.799824,1139
Starbucks,2.890080,1119
Chipotle Mexican Grill,3.043828,1118
Maskadores Taco Shop,4.417671,996


In [ ]:
sb_user_rating = starMatrix['Starbucks']
similar_to_sb = starMatrix.corrwith(sb_user_rating)
similar_to_sb.head()

/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:2526: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:2455: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


name
1 800 Flowers                   NaN
1 800-Flowers                   NaN
1 Brothers Pizza                NaN
1 One Armor                     NaN
1-800 WATER DAMAGE of Phoenix   NaN
dtype: float64

In [ ]:
# Creating dataframe to include # of ratings.
corr_sb = pd.DataFrame(similar_to_sb, columns=['Correlation'])
corr_sb.dropna(inplace=True)
corr_sb.head(30)

,Correlation
name,
16th Street Sports Bar & Grill,-0.408248
24 Carrots Juice Bar & Cafe,1.000000
24 Hour Fitness,-0.500000
A Bakeshop,0.666667
AAA Phoenix,1.000000
ADEQ Auto Emissions Inspection Station,1.000000
AK's Nails & Spa,-0.981981
AMC Ahwatukee 24,0.635135
AMC Westgate 20,0.686923


In [ ]:
#Bringining in ratings
corr_sb = corr_sb.join(ratings['number_of_ratings'])

corr_sb.head()

,Correlation,number_of_ratings
name,,
16th Street Sports Bar & Grill,-0.408248,52
24 Carrots Juice Bar & Cafe,1.000000,118
24 Hour Fitness,-0.500000,55
A Bakeshop,0.666667,189
AAA Phoenix,1.000000,127


In [ ]:
corr_sb[corr_sb['number_of_ratings'] > 100].sort_values(by='Correlation', ascending=False).head(10)

,Correlation,number_of_ratings
name,,
24 Carrots Juice Bar & Cafe,1.0,118
Pub'n Grub,1.0,112
Starbucks,1.0,1119
Sonic Drive-In,1.0,231
Sa Bai On The Fly,1.0,165
Bootleggers Modern American Smokehouse,1.0,200
Rania's Kabob Grill,1.0,344
Brisam's,1.0,180
Buff N File Nail Spa,1.0,285


In [46]:
# To html starbucks correlations
corr_sb6 = corr_sb[corr_sb['number_of_ratings'] > 100].sort_values(by='Correlation', ascending=False).head(6)
corr_sb6.drop(columns='Correlation')
corr_sb6.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Correlation</th>\n      <th>number_of_ratings</th>\n    </tr>\n    <tr>\n      <th>name</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>24 Carrots Juice Bar &amp; Cafe</th>\n      <td>1.0</td>\n      <td>118</td>\n    </tr>\n    <tr>\n      <th>Pub\'n Grub</th>\n      <td>1.0</td>\n      <td>112</td>\n    </tr>\n    <tr>\n      <th>Starbucks</th>\n      <td>1.0</td>\n      <td>1119</td>\n    </tr>\n    <tr>\n      <th>Sonic Drive-In</th>\n      <td>1.0</td>\n      <td>231</td>\n    </tr>\n    <tr>\n      <th>Sa Bai On The Fly</th>\n      <td>1.0</td>\n      <td>165</td>\n    </tr>\n    <tr>\n      <th>Bootleggers Modern American Smokehouse</th>\n      <td>1.0</td>\n      <td>200</td>\n    </tr>\n  </tbody>\n</table>'

In [ ]:
#Get chipotle recommendations

In [ ]:
ch_user_rating = starMatrix['Chipotle Mexican Grill']
similar_to_ch = starMatrix.corrwith(ch_user_rating)
similar_to_ch.head()

/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:2526: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:2455: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


name
1 800 Flowers                         NaN
1 800-Flowers                         NaN
1 Brothers Pizza                -0.738724
1 One Armor                           NaN
1-800 WATER DAMAGE of Phoenix         NaN
dtype: float64

In [ ]:
corr_ch = pd.DataFrame(similar_to_ch, columns=['Correlation'])
corr_ch.dropna(inplace=True)
corr_ch.head()

,Correlation
name,
1 Brothers Pizza,-0.738724
16th Street Sports Bar & Grill,-0.487500
24 Carrots Juice Bar & Cafe,-0.459532
5 & Diner,0.866025
7-Eleven,0.541736


In [ ]:
corr_ch = corr_ch.join(ratings['number_of_ratings'])

corr_ch.head()

,Correlation,number_of_ratings
name,,
1 Brothers Pizza,-0.738724,72
16th Street Sports Bar & Grill,-0.487500,52
24 Carrots Juice Bar & Cafe,-0.459532,118
5 & Diner,0.866025,6
7-Eleven,0.541736,29


In [47]:
# Chipotle top 6 to html.
corr_ch6 = corr_ch[corr_ch['number_of_ratings'] > 190].sort_values(by='Correlation', ascending=False).head(6)
corr_ch6 = corr_ch6.drop(columns='Correlation')
corr_ch6.to_html()


'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>number_of_ratings</th>\n    </tr>\n    <tr>\n      <th>name</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Pho Leo</th>\n      <td>288</td>\n    </tr>\n    <tr>\n      <th>Chipotle Mexican Grill</th>\n      <td>1118</td>\n    </tr>\n    <tr>\n      <th>Jalapeño Inferno</th>\n      <td>255</td>\n    </tr>\n    <tr>\n      <th>Tavern Grille Scottsdale</th>\n      <td>291</td>\n    </tr>\n    <tr>\n      <th>Earnhardt Toyota</th>\n      <td>285</td>\n    </tr>\n    <tr>\n      <th>Kabuki Sushi &amp; Grill</th>\n      <td>205</td>\n    </tr>\n  </tbody>\n</table>'

 Get Maskadores Taco Shop recommendations
 

In [32]:
Taco_user_rating = starMatrix['Maskadores Taco Shop']
similar_to_Taco = starMatrix.corrwith(Taco_user_rating)
similar_to_Taco.head()

/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:2526: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:2455: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


name
1 800 Flowers                   NaN
1 800-Flowers                   NaN
1 Brothers Pizza                NaN
1 One Armor                     NaN
1-800 WATER DAMAGE of Phoenix   NaN
dtype: float64

In [33]:
corr_Taco = pd.DataFrame(similar_to_Taco, columns=['Correlation'])
corr_Taco.dropna(inplace=True)
corr_Taco.head()

,Correlation
name,
30 Minute Alterations + Cleaning,1.000000
AMC Ahwatukee 24,0.277350
AMC Westgate 20,-0.618590
ATL Wings,-0.406181
AZ Food Crafters - Kitchen and Catering,1.000000


In [34]:
corr_Taco = corr_Taco.join(ratings['number_of_ratings'])

corr_Taco.head()

,Correlation,number_of_ratings
name,,
30 Minute Alterations + Cleaning,1.000000,78
AMC Ahwatukee 24,0.277350,156
AMC Westgate 20,-0.618590,208
ATL Wings,-0.406181,238
AZ Food Crafters - Kitchen and Catering,1.000000,114


In [40]:
corr_taco5 = corr_Taco[corr_Taco['number_of_ratings'] > 190].sort_values(by='Correlation', ascending=False).head(5)
corr_taco5.head()


,Correlation,number_of_ratings
name,,
Macy's,1.0,286
Buff N File Nail Spa,1.0,285
Giuseppe's On 28th,1.0,419
Luci's at the Orchard,1.0,438
Maskadores Taco Shop,1.0,996


In [43]:
# html table output for Taco place. Dropping correlation column.
corr_taco5 = corr_taco5.drop(columns='Correlation')
corr_taco5.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>number_of_ratings</th>\n    </tr>\n    <tr>\n      <th>name</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Macy\'s</th>\n      <td>286</td>\n    </tr>\n    <tr>\n      <th>Buff N File Nail Spa</th>\n      <td>285</td>\n    </tr>\n    <tr>\n      <th>Giuseppe\'s On 28th</th>\n      <td>419</td>\n    </tr>\n    <tr>\n      <th>Luci\'s at the Orchard</th>\n      <td>438</td>\n    </tr>\n    <tr>\n      <th>Maskadores Taco Shop</th>\n      <td>996</td>\n    </tr>\n  </tbody>\n</table>'

In [36]:
TacoTable = corr_Taco[corr_Taco['number_of_ratings'] > 190].sort_values(by='Correlation', ascending=False).head(10)
TacoTable = TacoTable.merge(dfUsers,on='name',left_on=None, right_on=None)
TacoTable.head(10)

,user_id,ratings_performed,business_id,name,address,city,state,postal_code,stars_x,Correlation,number_of_ratings
0,ELcQDlf69kb-ihJfxZyL0A,238,OD09pm-MzynigAvD5KoGPA,Maskadores Taco Shop,818 W Broadway Rd,Tempe,AZ,85282.0,3.0,1.0,996
1,ELcQDlf69kb-ihJfxZyL0A,238,OD09pm-MzynigAvD5KoGPA,Maskadores Taco Shop,818 W Broadway Rd,Tempe,AZ,85282.0,5.0,1.0,996
2,DK57YibC5ShBmqQl97CKog,168,I7ZzsHeuf1IQyrG3JyECmg,Maskadores Taco Shop,"1721 E Warner Rd, Ste C1B",Tempe,AZ,85284.0,3.0,1.0,996
3,DK57YibC5ShBmqQl97CKog,168,OD09pm-MzynigAvD5KoGPA,Maskadores Taco Shop,818 W Broadway Rd,Tempe,AZ,85282.0,4.0,1.0,996
4,PcvbBOCOcs6_suRDH7TSTg,122,OD09pm-MzynigAvD5KoGPA,Maskadores Taco Shop,818 W Broadway Rd,Tempe,AZ,85282.0,2.0,1.0,996
5,bhJ6ivAuSpgXP4JiKWjSZQ,100,OD09pm-MzynigAvD5KoGPA,Maskadores Taco Shop,818 W Broadway Rd,Tempe,AZ,85282.0,3.0,1.0,996
6,bpgbVPJcG-wQ4cNBcPPIxw,91,I7ZzsHeuf1IQyrG3JyECmg,Maskadores Taco Shop,"1721 E Warner Rd, Ste C1B",Tempe,AZ,85284.0,5.0,1.0,996
7,bpgbVPJcG-wQ4cNBcPPIxw,91,OD09pm-MzynigAvD5KoGPA,Maskadores Taco Shop,818 W Broadway Rd,Tempe,AZ,85282.0,4.0,1.0,996
8,SRK_sUnvRmMwfexoL_DLQA,82,OD09pm-MzynigAvD5KoGPA,Maskadores Taco Shop,818 W Broadway Rd,Tempe,AZ,85282.0,4.0,1.0,996
9,zFYs8gSUYDvXkb6O7YkRkw,71,I7ZzsHeuf1IQyrG3JyECmg,Maskadores Taco Shop,"1721 E Warner Rd, Ste C1B",Tempe,AZ,85284.0,5.0,1.0,996


In [ ]:
# AZ data sorted by most to least ratings performed by user.
dfAZ.head()

# AZ data sorted by most to least ratings performed by user.
dfAZ = dfAZ.merge(ratingsCount, on='user_id')
dfAZ.head()

ratingsCount = ratingsCount.loc[:, ['user_id', 'ratings_performed']]